# Introduction to gridding and working with grids
A common activity for working with scanning radar data is the estimation of the radar measuremets (and retrievals) on a regularly spaced grid. This could be for GIS purposes, the compraring to other data sources or for comparisons to model outputs. 

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import pyart
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature


import matplotlib.ticker as mticker



In [ ]:
#Lets read in our data set
radar = pyart.io.read(os.path.expanduser('~/data/arm/csapr_test_case.nc'), 
                      delay_field_loading = True)


## Lets get some basic geometry about our radar data

In [ ]:
max_lat = radar.gate_latitude['data'].max()
min_lat = radar.gate_latitude['data'].min()
max_lon = radar.gate_longitude['data'].max()
min_lon = radar.gate_longitude['data'].min()
print(max_lat, min_lat, max_lon, min_lon)

height = radar.gate_altitude
lats = radar.gate_latitude
lons = radar.gate_longitude
lat_lines = np.arange(min_lat, max_lat, .5)
lon_lines = np.arange(min_lon, max_lon, .5)


## Lets take a look at all the bells and whistles on Py-ART's Gridder

In [ ]:
pyart.map.map_gates_to_grid?

## Lets make a regular grid 
31 vertical levels, 301 by 301 horizontal points. 0 to 15km in height, and -120 to 120km in x and y. 

In [ ]:
grids = pyart.map.grid_from_radars(radar,(31,301,301),
                   ((0.,15000.),(-120000.,120000.),(-120000.,120000.)),
                   refl_field='reflectivity', weighting_function='Barnes2')

# So what did we get???

In [ ]:
grids?

In [ ]:
grids.fields.keys()

## Why your own grid object? What about xarray?
When Py-ART was being developed xarray was very new.. but fear not! We have a simple method for converting

In [ ]:
grids.to_xarray?

In [ ]:
xgrids = grids.to_xarray()


In [ ]:
xgrids

In [ ]:
xgrids.reflectivity.mean()

In [ ]:
xgrids.reflectivity.sel(z=500, time=xgrids.time[0])

In [ ]:
xgrids.reflectivity.sel(z=500, time=xgrids.time[0]).plot.pcolormesh(cmap=pyart.graph.cm_colorblind.HomeyerRainbow, vmin=-6, vmax=64)

In [ ]:
fig = plt.figure(figsize=[15, 7])
ax = plt.axes(projection=ccrs.LambertConformal())
pc = xgrids.reflectivity.sel(z=3000, 
                        time=xgrids.time[0]).plot.pcolormesh(cmap=pyart.graph.cm_colorblind.HomeyerRainbow, 
                                                             vmin=-6, vmax=64)
gl = ax.gridlines(draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')

In [ ]:
fig = plt.figure(figsize=[15, 7])
pc = xgrids.reflectivity.sel(x=0, time=xgrids.time[0]).plot.pcolormesh(cmap=pyart.graph.cm_colorblind.HomeyerRainbow,
                                                                      vmin=-6, vmax=64)



In [ ]:
# A little trick with much thanks to Ryan Abernathy

lalogrids = xgrids.swap_dims({"x": "lon"})
lalogrids = lalogrids.swap_dims({"y": "lat"})


In [ ]:
#lets make sure we get our projection right!

print(grids.projection)
lon_0 = grids.get_projparams()['lon_0']
lat_0 = grids.get_projparams()['lat_0']
proj = ccrs.AzimuthalEquidistant(central_latitude=lat_0, central_longitude=lon_0)


fig = plt.figure(figsize=[15, 7])
display = pyart.graph.GridMapDisplay(grids)

# panel sizes
map_panel_axes = [0.05, 0.05, .4, .80]
x_cut_panel_axes = [0.55, 0.10, .4, .25]
y_cut_panel_axes = [0.55, 0.50, .4, .25]

# panel 1
ax1 = fig.add_axes(map_panel_axes, projection=proj)


pc = xgrids.reflectivity.sel(z=3000, 
                        time=xgrids.time[0]).plot.pcolormesh(cmap=pyart.graph.cm_colorblind.HomeyerRainbow, 
                                                             vmin=-6, vmax=64, ax = ax1)
gl = ax1.gridlines(draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')

ax1.add_feature(cfeature.RIVERS)
ax1.add_feature(cfeature.STATES)


#ax.add_feature(cartopy.feature.NaturalEarthFeature('cultural', 'roads', '10m'))
roads = cfeature.NaturalEarthFeature(
    category='cultural',
    name='roads',
    scale='10m',
    facecolor='none',
    edgecolor='black',
    alpha=0.5)

ax1.add_feature(roads)

ax1.add_feature(cfeature.LAKES, zorder=0)


# panel 2, along longitude slice.
ax2 = fig.add_axes(x_cut_panel_axes)

pc = lalogrids.reflectivity.sel(lat=36.9, time=xgrids.time[0],method="nearest").plot.pcolormesh(cmap=pyart.graph.cm_colorblind.HomeyerRainbow,
                                                                      vmin=-6, vmax=64, ax=ax2)

ax2.set_ylim([0,15000])
#ax2.set_xlim([-120000,120000])
ax2.set_xlabel('Distance from SGP CF (km)')

# panel 3, along latitude slice
ax3 = fig.add_axes(y_cut_panel_axes)

pc = lalogrids.reflectivity.sel(lon=-97.45, time=xgrids.time[0],method="nearest").plot.pcolormesh(cmap=pyart.graph.cm_colorblind.HomeyerRainbow,
                                                                      vmin=-6, vmax=64, ax=ax3)

ax3.set_ylim([0,15000])
#ax3.set_xlim([-120000,120000])
ax3.set_xlabel('Distance from SGP CF (km)')


In [ ]:
grids.